In [1]:
import os
%pwd
os.chdir('../')

In [2]:
%pwd

'c:\\Data-Science\\ML Projects\\End_to_End_ML_Project'

### Entity


In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

### Configuration Manager


In [5]:
from src.e2emlproject.constants import *
from src.e2emlproject.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_file_path=CONFIG_FILE_PATH,
        params_file_path=PARAMS_FILE_PATH,
        schema_file_path=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifcats_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
        )
        return model_evaluation_config

### Components


In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from src.e2emlproject.utils.common import save_json
import joblib


class ModelEvaluator:

    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        y_pred = model.predict(test_x)

        (rmse, mae, r2) = self.eval_metrics(test_y, y_pred)

        scores = {"RMSE": rmse, "MAE": mae, "R^2": r2}
        save_json(path=Path(self.config.metric_file_name), data=scores)

### Pipeline


In [12]:
from src.e2emlproject.logging import logger

try:
    logger.info("Model Evaluation process started!:")
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluator(config=model_evaluation_config)
    model_evaluation.save_results()
    logger.info("Model Evaluation process completed!")
except Exception as e:
    raise e

[2024-05-05 18:18:08,394 INFO 3502542471 - Model Evaluation process started!:]
[2024-05-05 18:18:08,398 INFO common - yaml file: config\config.yaml loaded successfully]
[2024-05-05 18:18:08,399 INFO common - yaml file: params.yaml loaded successfully]
[2024-05-05 18:18:08,401 INFO common - yaml file: schema.yaml loaded successfully]
[2024-05-05 18:18:08,403 INFO common - created directory at: artifacts]
[2024-05-05 18:18:08,404 INFO common - created directory at: artifacts/model_evaluator]
[2024-05-05 18:18:08,416 INFO common - json file saved at: artifacts\model_evaluator\metrics.json]
[2024-05-05 18:18:08,417 INFO 3502542471 - Model Evaluation process completed!]
